In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
import seaborn as sns
import random

import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

from arch import arch_model

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t
from scipy.optimize import minimize
from scipy.stats import ncx2

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

import pymc as pm
import pytensor.tensor as pt
import arviz as az

import tensorflow as tf
from tensorflow import keras


#from tensorflow.keras.utils import plot_model


######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class CIRModel:
    def __init__(self, df, log_return, nfuture):
        self.df = df
        self.series = log_return
        self.kappa = None
        self.theta = None
        self.sigma = None
        self.v0 = None
        self.nfuture = nfuture
        self.trading_days = 252
        self.dt = 1 / self.trading_days
        self.rolling_rv = None

        self.rng = np.random.default_rng(42)

    def realized_volatility(self):
        # Rolling realized volatility
        self.df['log_return'] = np.log(self.df['Close'] / self.df['Close'].shift(1))
        self.rolling_rv = self.df['log_return'].groupby(self.df.index.date).apply(lambda x: np.sqrt(np.sum((x - x.mean())**2)))
        self.rolling_rv.dropna(inplace=True)

        self.rolling_rv_sq = self.rolling_rv ** 2
        self.v0 = self.rolling_rv_sq.iloc[-self.nfuture]
        self.params_regression()
        #self.params_fit()

    def params_regression(self):
        v = self.rolling_rv_sq.values
        v_lag = v[:-1]   # v_t
        v_next = v[1:]   # v_{t+1}

        # Regression: v_next = a + b * v_lag + error #Δvt​≈κ(θ−vt​)Δt+σvt​Δt --> vt+1​=(1−κΔt)vt​+κθΔt+error
        X = sm.add_constant(v_lag)
        model = sm.OLS(v_next, X).fit()
        a, b = model.params

        # Parameters
        self.kappa = -np.log(b) / self.dt
        self.theta = a / (self.kappa * self.dt)

        # Residuals for sigma
        residuals = v_next - (a + b * v_lag)
        var_resid = np.var(residuals)

        self.sigma = np.sqrt(var_resid / (self.dt * np.mean(v)))

        return self.kappa, self.theta, self.sigma

    def simulate_exact(self, n_steps):
        v_paths = np.zeros((self.nfuture + 1, n_steps))
        v_paths[0] = self.v0

        for t in range(1, self.nfuture + 1):
            v_prev = v_paths[t-1]
            exp_kdt = np.exp(-self.kappa * dt)
            c = (self.sigma**2 * (1 - exp_kdt)) / (4 * self.kappa)
            d = 4 * self.kappa * self.theta / (self.sigma**2)
            lam = (4 * self.kappa * exp_kdt * v_prev) / (self.sigma**2 * (1 - exp_kdt))
            # sample from noncentral chi-square
            x = ncx2.rvs(d, lam, size=n_steps)
            v_paths[t] = c * x

        return v_paths

    def simulate_euler(self, n_steps):
        dt = 1 / self.trading_days
        v_paths = np.zeros((self.nfuture + 1, n_steps))
        v_paths[0] = self.v0

        for t in range(1, self.nfuture + 1):
            v_prev = v_paths[t-1]
            # Brownian increments
            dW = np.random.normal(0, np.sqrt(dt), size=n_steps)
            # Euler-Maruyama update
            v_new = v_prev + self.kappa * (self.theta - v_prev) * dt + self.sigma * np.sqrt(np.maximum(v_prev, 0)) * dW
            v_paths[t] = np.maximum(v_new, 0)  # enforce positivity

        return v_paths

    def forecast_volatility(self, n_steps, alpha):
        """Plot realized vs forecast with CI and forecast error."""
        v_paths = self.simulate_euler(n_steps)

        # convert variance paths -> volatility (annualized)
        vol_paths = np.sqrt(v_paths) * np.sqrt(self.trading_days)

        # statistics across simulations
        forecast_mean = vol_paths.mean(axis=1)
        forecast_ci_lower = np.percentile(vol_paths, 100 * (alpha / 2), axis=1)
        forecast_ci_upper = np.percentile(vol_paths, 100 * (1 - alpha / 2), axis=1)

        forecast_index = self.rolling_rv.index[-self.nfuture-1:]
        forecast_series = pd.Series(forecast_mean, index=forecast_index)
        forecast_lower = pd.Series(forecast_ci_lower, index=forecast_index)
        forecast_upper = pd.Series(forecast_ci_upper, index=forecast_index)

        # realized volatility (annualized)
        realized_vol = self.rolling_rv * np.sqrt(self.trading_days)
        realized_vals = realized_vol.loc[forecast_series.index]

        # --- Forecast errors ---
        # point forecast error
        forecast_error = (forecast_series - realized_vals) / realized_vals
        mae = np.mean(np.abs(forecast_error))
        rmse = np.sqrt(np.mean(forecast_error**2))

        # CI coverage: % of realized values inside forecast CI
        inside_band = ((realized_vals >= forecast_lower) &
                      (realized_vals <= forecast_upper)).mean()
        coverage_error = 1 - inside_band  # miss rate

        # Plotting
        fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
        start_index = -self.nfuture - 21
        end_index = -1

        # Historical + Forecast
        axs[0].plot(realized_vol, color='black', alpha=1, label='Realized Vol')
        axs[0].plot(forecast_series, color='red', linestyle='--', label='CIR Forecast (Mean)')
        axs[0].fill_between(forecast_series.index, forecast_lower, forecast_upper,
                            color='blue', alpha=0.3, label=f'{100*(1-alpha):.0f}% CI')
        axs[0].axvline(x=forecast_series.index[0], color='black', linestyle='--', linewidth=1.5)
        axs[0].set_title(
            f'Historical Realized Volatility and CIR Forecast\n'
            f'MAE={mae:.4f}, RMSE={rmse:.4f}, Coverage Error={coverage_error:.2%}'
        )
        axs[0].set_ylabel('Annualized Volatility')
        axs[0].legend()
        axs[0].grid(True)
        axs[0].set_xlim([realized_vol.index[start_index], realized_vol.index[end_index]])
        axs[0].set_ylim(0, np.amax(forecast_series) * 2)

        # Forecast errors plot
        axs[1].bar(forecast_error.index, forecast_error.values * 100, color='purple')
        axs[1].plot(forecast_error * 100, marker='o', linestyle='--', color='purple', alpha=0.7)
        axs[1].axhline(0, color='black', linestyle='-')
        axs[1].axvline(x=forecast_series.index[0], color='black', linestyle='--', linewidth=1.5)
        axs[1].set_title('Relative Forecast Error (Forecast - Realized) / Realized')
        axs[1].set_ylabel('Relative Error')
        axs[1].grid(True)

        plt.tight_layout()

In [ ]:
##CIR --> by fitting non-central chi-squared distribution

import numpy as np
import scipy.optimize as opt
from scipy.stats import ncx2

def cir_negative_loglik(params, v_series, dt):
    """
    Negative log-likelihood for CIR using exact transition density.
    params: array-like [kappa, theta, sigma] (all > 0)
    v_series: 1D numpy array of realized variances v_t
    dt: time step in years (e.g., 1/252)
    """
    kappa, theta, sigma = params

    # enforce positivity in function (minimizer may propose negatives)
    if kappa <= 0 or theta <= 0 or sigma <= 0:
        return 1e10

    # precompute
    exp_kdt = np.exp(-kappa * dt)
    one_minus = 1.0 - exp_kdt
    # small guard
    if one_minus <= 0 or sigma <= 0 or kappa <= 0:
        return 1e10

    c = (sigma**2 * one_minus) / (4.0 * kappa)
    d = (4.0 * kappa * theta) / (sigma**2)

    v = v_series
    v_t = v[:-1]
    v_tp1 = v[1:]

    # avoid zeros
    eps = 1e-16
    v_t = np.maximum(v_t, eps)
    v_tp1 = np.maximum(v_tp1, eps)

    # noncentrality parameter
    lam = (4.0 * kappa * exp_kdt * v_t) / (sigma**2 * one_minus)

    # scaled variable for ncx2: x = v_{t+1} / c
    x = v_tp1 / c

    # compute logpdf of non-central chi-square at x with d dof and lambda
    # logpdf for scaled variable: log f_V(v_tp1) = logpdf_chi2(x) - log(c)
    # use scipy.stats.ncx2.logpdf
    with np.errstate(divide='ignore', invalid='ignore'):
        logpdf_vals = ncx2.logpdf(x, df=d, nc=lam)
        # subtract log(c) due to scale
        logpdf_vals = logpdf_vals - np.log(c)

    # handle -inf or nan (very unlikely if x>0)
    if np.any(~np.isfinite(logpdf_vals)):
        # give large penalty
        return 1e10

    neg_ll = -np.sum(logpdf_vals)
    return neg_ll

def estimate_cir_mle(v_series, dt=1/252, initial_guess=None, bounds=None, enforce_feller=False):
    """
    v_series: 1D array-like of realized variances (length T)
    dt: timestep in years
    initial_guess: optional [kappa, theta, sigma], else derived from OLS
    bounds: optional list of (min,max) for kappa,theta,sigma
    enforce_feller: if True, add constraint 2*kappa*theta >= sigma^2 via penalty
    """
    v = np.asarray(v_series)
    if v.ndim != 1 or v.size < 5:
        raise ValueError("v_series must be 1D with at least several observations")

    # default initial guess from regression (quick method)
    if initial_guess is None:
        # simple OLS to get slope/intercept as earlier
        v_lag = v[:-1]
        v_next = v[1:]
        A = np.vstack([np.ones_like(v_lag), v_lag]).T
        beta, *_ = np.linalg.lstsq(A, v_next, rcond=None)
        a, b = beta  # v_next ≈ a + b * v_lag
        # transform to CIR approximate
        kappa_init = max(1e-6, -np.log(b) * (1.0/dt)) if b>0 else 1.0
        theta_init = max(1e-8, a / (kappa_init * dt)) if kappa_init>0 else np.mean(v)
        # residual-based sigma initial
        resid = v_next - (a + b * v_lag)
        var_resid = np.var(resid, ddof=1)
        sigma_init = np.sqrt(max(var_resid / (dt * np.mean(v)), 1e-8))
        initial_guess = np.array([kappa_init, theta_init, sigma_init])

    # bounds default
    if bounds is None:
        bounds = [(1e-8, 50.0),    # kappa
                  (1e-12, np.max(v)*10 + 1.0),  # theta
                  (1e-8, 5.0)]     # sigma

    # objective with optional feller penalty
    def obj(par):
        val = cir_negative_loglik(par, v, dt)
        if enforce_feller:
            k, th, s = par
            penalty = 0.0
            if 2.0*k*th < s*s:
                # quadratic penalty
                gap = s*s - 2.0*k*th
                penalty = 1e6 * gap**2
            val = val + penalty
        return val

    result = opt.minimize(obj, x0=initial_guess, bounds=bounds, method='L-BFGS-B',
                          options={'disp': False, 'maxiter': 1000})

    if not result.success:
        # you may still return the best found
        # print("Optimization did not converge:", result.message)
        pass

    est = result.x
    return {'kappa': float(est[0]), 'theta': float(est[1]), 'sigma': float(est[2]),
            'success': result.success, 'message': result.message, 'initial_guess': initial_guess}

# Example usage:
# v_series = cir_calibration.rolling_rv_sq.values  # from your realized_volatility() routine
# res = estimate_cir_mle(v_series, dt=1/252)
# print(res)
